In [4]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open("demo/img/cataract.png")).unsqueeze(0).to(device)
text = clip.tokenize(["no cataract", "mild cataract", "severe cataract"]).to(device)

with torch.no_grad():


    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)

Label probs: [[0.24   0.2676 0.4924]]


In [6]:
import os
import clip
import torch

import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
from tqdm import tqdm
from dataset import MyDataset
import pandas as pd
data_dir = 'D:\\PycharmProjects\\fairness\\pics\\cataract\\all_pic\\'
# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Load the dataset
csv = pd.read_csv(os.path.join(data_dir, 'train.csv'))

valid_dataset = MyDataset(data_dir, img_size=64, transform=None, train=False)
train_dataset = MyDataset(data_dir, img_size=64, transform=None, train=True, csv_file=csv)
def get_features(dataset):
    all_features = []
    all_labels = []

    with torch.no_grad():
        for images, labels, _ in tqdm(DataLoader(dataset, batch_size=16)):
            features = model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

# Calculate the image features
train_features, train_labels = get_features(train_dataset)
test_features, test_labels = get_features(valid_dataset)

# Perform logistic regression
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
classifier.fit(train_features, train_labels)

# Evaluate using the logistic regression classifier
predictions = classifier.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(float)) * 100.
print(f"Accuracy = {accuracy:.3f}")

  0%|          | 0/29 [00:01<?, ?it/s]


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [768, 3, 32, 32], but got 3-dimensional input of size [16, 64, 64] instead